# Análise de dados com SQL

### De onde vem o dataset?
O dataset usado é da competição hospedada no Kaggle [Predict Future Sales](https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data). Consiste no histórico de vendas de algumas lojas na rússia entre Janeiro de 2013 e Outubro de 2015, provida pela [1C Company](https://1c.ru/eng/title.htm) para uma atividade no curso ["How to win a data science competition"](https://www.coursera.org/learn/competitive-data-science/home/welcome) na Coursera.

### Como está dividido esse documento?
Na primeira parte, o arquivo de banco de dados **SQLite** é criado no arquivo `sales.db`, e as tabelas construídas a partir da modelagem feita na imagem abaixo. A biblioteca Pandas é utilizada para processar os arquivos CSV e dividir os dados entre as colunas do banco.

![modeling](https://github.com/alissone/banco_de_dados/raw/main/base_modeling.svg)

Na segunda parte o arquivo é alternativamente lido de um banco já preenchido e em seguida cada query é mostrada em funcionamento, conforme o jupyter notebook na raiz do repositório.

In [19]:
import os
from pathlib import Path
import sqlite3
import pandas as pd

CSV_FOLDER = 'kaggle_download/'
QUERIES_FOLDER = 'queries/'

def criar_banco(filename='sales.db'):
    try:
        os.remove(filename)
    except Exception:
        pass

    Path('sales.db').touch()

def criar_tabelas(filename='sales.db'):
    conn = sqlite3.connect(filename)
    c = conn.cursor()


    c.execute('''
    CREATE TABLE item_categories (
        item_category_name VARCHAR(20),
        item_category_id INT(5),
        PRIMARY KEY (item_category_id)
    );
    ''')
    
    c.execute('''
    CREATE TABLE items (
        item_id INT(5),
        item_name VARCHAR(40) DEFAULT '',
        item_category_id INT(5),
        FOREIGN KEY(item_category_id) REFERENCES item_categories(item_category_id),
        PRIMARY KEY (item_id)
    );
    ''')
    
    c.execute('''
    CREATE TABLE shops (
        shop_id INT(5),
        shop_name VARCHAR(40) DEFAULT '',
        PRIMARY KEY (shop_id)
    );
    ''')
    
    
    c.execute('''
    CREATE TABLE sales (
        sale_id INT(5) NOT NULL,
        item_id INT(5),
        date DATE(10),
        item_price DOUBLE(20),
        item_cnt_day INT(5),
        shop_id INT(5),
        FOREIGN KEY(item_id) REFERENCES items(item_id),
        FOREIGN KEY(shop_id) REFERENCES shops(shop_id),
        PRIMARY KEY (sale_id)
    );
    ''')


    shops = pd.read_csv(CSV_FOLDER + 'shops.csv')
    shops.to_sql('shops', conn, if_exists='append', index = False)
    
    items = pd.read_csv(CSV_FOLDER + 'items.csv')
    items.to_sql('items', conn, if_exists='append', index = False)
    
    sales = pd.read_csv(CSV_FOLDER + 'sales_train.csv')
    sales['date'] = pd.to_datetime(sales['date'], format='%d.%m.%Y') # A data vem no formato russo, é necessário converter para ISO
    sales = sales.drop('date_block_num', 1) # Date block num é uma coluna redundante com a mesma informação que tem em `date`
    sales = sales.rename_axis('sale_id').reset_index() # SQLite3 não suporta autoincrement, então criamos uma coluna de id manualmente
    
    sales.to_sql('sales', conn, if_exists='append', index = False)
    
    item_categories = pd.read_csv(CSV_FOLDER + 'item_categories.csv')
    item_categories.to_sql('item_categories', conn, if_exists='append', index = False)
    conn.close()



In [20]:
arquivo_banco = 'sales.db'
criar_banco(arquivo_banco)
criar_tabelas(arquivo_banco)

conn = sqlite3.connect(arquivo_banco)
c = conn.cursor()

HA


In [22]:
def ler_arquivo(arquivo):
    with open(arquivo, "r") as db:
        lines = list(db.readlines())

        return "\n".join(lines)

## Query 1 - Lista de jogos de PC mais vendidos em todas as lojas

![query1](https://github.com/alissone/banco_de_dados/raw/main/queries/query1.png)

In [ ]:
query1 = ler_arquivo(QUERIES_FOLDER + "query1.sql")
pd.read_sql(query1, conn)

,Vendas,Nome,Preço Médio,Categoria
0,17245,"Diablo III [PC, Jewel, русская версия]",₽ 999.58,Игры PC - Стандартные издания
1,10099,"Grand Theft Auto V [PC, русские субтитры]",₽ 2051.76,Игры PC - Стандартные издания
2,10032,"Battlefield 4 [PC, русская версия]",₽ 922.22,Игры PC - Стандартные издания
3,9012,World of Warcraft. Карта оплаты игрового време...,₽ 808.54,Игры PC - Дополнительные издания
4,8262,"FIFA 14 [PC, русская версия]",₽ 1018.78,Игры PC - Стандартные издания
5,7788,"Crysis 3 [PC, русская версия]",₽ 636.41,Игры PC - Стандартные издания
6,7588,"Call of Duty: Black Ops II [PС, Jewel, русская...",₽ 420.24,Игры PC - Стандартные издания
7,7351,StarCraft II: Heart of the Swarm (дополнение) ...,₽ 644.06,Игры PC - Стандартные издания
8,7105,"Diablo III: Reaper of Souls (дополнение) [PC, ...",₽ 888.23,Игры PC - Дополнительные издания
9,6975,"Battlefield 3 [PC, Jewel, русская версия]",₽ 528.72,Игры PC - Стандартные издания


## Query 2 - Lista de lojas com mais de 5000 produtos anunciados e seu item mais vendido

![query2](https://github.com/alissone/banco_de_dados/raw/main/queries/query2.png)

In [24]:
query2 = ler_arquivo(QUERIES_FOLDER + "query2.sql")
pd.read_sql(query2, conn)

,Itens a venda,Nome da Loja,Item mais vendido
0,235636,"Москва ТЦ ""Семеновский""",ЖЕНИТЬБА БАЛЬЗАМИНОВА (rem)
1,186104,"Москва ТРК ""Атриум""",V/A The Golden Jazz Collection 1 2CD
2,143480,"Химки ТЦ ""Мега""","Assassin's Creed: Изгой [PC, русская версия]"
3,142234,"Москва ТЦ ""МЕГА Теплый Стан"" II",Фирменный пакет майка 1С Интерес белый (34*42)...
4,117428,"Якутск Орджоникидзе, 56",ЩЕНЯЧИЙ ПАТРУЛЬ Сезон 1 Выпуск 1 Гав - гав ...
5,109253,"СПб ТК ""Невский Центр""",ТРОФИМОВ СЕРГЕЙ Черное и белое (фирм.)
6,105366,"Москва ТЦ ""МЕГА Белая Дача II""","Билет ""ИгроМир 2015"" - 3 октября 2015 [Цифрова..."
7,82663,"Воронеж (Плехановская, 13)",Брелок металлический WOT Колобанов (100104)
8,71441,"Якутск ТЦ ""Центральный""","Batman: Рыцарь Аркхема [PC, Jewel, русские суб..."
9,69573,"Чехов ТРЦ ""Карнавал""",DEPECHE MODE Black Celebration LP


## Query 3 - Número de vendas e preço por mês do jogo "Dishonred Game of the Year Edition" para XBOX 360 durante o ano de 2014


![query3](https://github.com/alissone/banco_de_dados/raw/main/queries/query3.png)

In [ ]:
query3 = ler_arquivo(QUERIES_FOLDER + "query3.sql")
pd.read_sql(query3, conn)


,Vendas,Mês,Preço Médio,Nome do item,Categoria
0,18,Jan,₽ 1776.78,"Dishonored Game of the Year Edition [Xbox 360,...",Игры - XBOX 360
1,26,Fev,₽ 1321.33,"Dishonored Game of the Year Edition [Xbox 360,...",Игры - XBOX 360
2,27,Mar,₽ 1259.01,"Dishonored Game of the Year Edition [Xbox 360,...",Игры - XBOX 360
3,20,Abr,₽ 1286.01,"Dishonored Game of the Year Edition [Xbox 360,...",Игры - XBOX 360
4,13,Mai,₽ 1573.95,"Dishonored Game of the Year Edition [Xbox 360,...",Игры - XBOX 360
5,10,Jul,₽ 1259.06,"Dishonored Game of the Year Edition [Xbox 360,...",Игры - XBOX 360
6,12,Ago,₽ 1336.19,"Dishonored Game of the Year Edition [Xbox 360,...",Игры - XBOX 360
7,1,Set,₽ 1259.00,"Dishonored Game of the Year Edition [Xbox 360,...",Игры - XBOX 360
8,3,Out,₽ 1259.20,"Dishonored Game of the Year Edition [Xbox 360,...",Игры - XBOX 360
9,2,Nov,₽ 1259.00,"Dishonored Game of the Year Edition [Xbox 360,...",Игры - XBOX 360


## Query 4 - Top filmes e coletâneas em DVD mais caros


![query4](https://github.com/alissone/banco_de_dados/raw/main/queries/query4.png)

In [23]:
query4 = ler_arquivo(QUERIES_FOLDER + "query4.sql")
pd.read_sql(query4, conn)


,Item,Data da venda,Preço Médio,Categoria
0,ФИЛЬМЫ ВЛАДИМИРА МОТЫЛЯ 5DVD (rem),04/07/2015,₽ 1399.00,Кино - DVD
1,ТАРКОВСКИЙ АНДРЕЙ 5DVD (rem),17/10/2015,₽ 1299.00,Кино - DVD
2,ЛУНТИК. СЕЗОН 6 (5 DVD),12/12/2013,₽ 987.23,Кино - DVD
3,ВАРИАНТ ОМЕГА 3DVD (rem),26/08/2015,₽ 899.00,Кино - DVD
4,"ВЕСТЕРН ПО ИТАЛЬЯНСКИ ""СПАГЕТТИ...""(3 dvd)",05/07/2014,₽ 899.00,Кино - DVD
5,МОЙ МАЛЕНЬКИЙ ПОНИ Подарочный набор,23/10/2015,₽ 898.85,Кино - DVD
6,ВВС:ЧУДЕСА СОЛНЕЧНОЙ СИСТЕМЫ (5 DVD),05/02/2015,₽ 832.82,Кино - DVD
7,ВВС:ЖИЗНЬ (5DVD),21/12/2013,₽ 832.37,Кино - DVD
8,ВВС:80 ЧУДЕС СВЕТА (5dvd),03/06/2014,₽ 830.41,Кино - DVD
9,ВВС: ТАЙНЫ ТИХОГО ОКЕАНА 5DVD,08/04/2015,₽ 823.56,Кино - DVD
